In [9]:
import numpy as np
import pandas as pd
from transformers import pipeline

import nltk
nltk.download('vader_lexicon')

import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\MuhammadAulia\AppData\Roaming\nltk_data...


In [3]:
df = pd.read_csv('data/hasil_preprocessingData.csv')
df

,Message
0,siap siap buat isi full tank bro
1,promo tebar undi apa bisa ikut
2,oke siap noted
3,buat guna baru aja sih
4,ayo ayo
...,...
6414,wow promo spesial akhir tahun
6415,kalo gue pake bakal keren gak
6416,min abis tukar ambil hadiah
6417,loh iya besok tahun baru


In [4]:
# Reset nomor baris data
df = df.reset_index(drop=True)

In [6]:
# Pakai GPU
classifier = pipeline("sentiment-analysis", model="w11wo/indonesian-roberta-base-sentiment-classifier", device=-1)

# Batch prediction
batch_size = 64
hasil_label = []

for i in range(0, len(df), batch_size):
    teks_batch = df['Message'].iloc[i:i+batch_size].tolist()
    pred_batch = classifier(teks_batch)
    hasil_label.extend([x['label'] for x in pred_batch])

df['sentimen_bert'] = hasil_label

Device set to use cpu


In [13]:
# Inisialisasi VADER
sid = SentimentIntensityAnalyzer()

# Fungsi klasifikasi sentimen dengan VADER
def vader_sentiment(text):
    scores = sid.polarity_scores(text)
    compound = scores['compound']
    if compound >= 0.05:
        return 'positive'
    elif compound <= -0.05:
        return 'negative'
    else:
        return 'neutral'

# Terapkan ke kolom cleaned_text
df['sentimen_vader'] = df['Message'].apply(vader_sentiment)


In [16]:
df

,Message,sentimen_bert,sentimen_vader
0,siap siap buat isi full tank bro,positive,neutral
1,promo tebar undi apa bisa ikut,neutral,neutral
2,oke siap noted,negative,neutral
3,buat guna baru aja sih,negative,neutral
4,ayo ayo,positive,neutral
...,...,...,...
6414,wow promo spesial akhir tahun,positive,positive
6415,kalo gue pake bakal keren gak,negative,neutral
6416,min abis tukar ambil hadiah,neutral,neutral
6417,loh iya besok tahun baru,negative,neutral


In [ ]:
# df.to_csv('data/dataSentimen.csv', index=False)